### Sea Ice Diagnostics for two CESM3 runs

In [ ]:
import xarray as xr
import numpy as np
import yaml

In [ ]:
CESM_output_dir = "/glade/campaign/cesm/development/cross-wg/diagnostic_framework/CESM_output_for_testing"
serial = False  # use dask LocalCluster

case_name = "b.e30_beta04.BLT1850.ne30_t232_wgx3.121congel"
base_case_name = "b.e30_beta04.BLT1850.ne30_t232_wgx3.121"

start_date = "0001-01-01"
end_date = "0100-01-01"
base_start_date = "0001-01-01"
base_end_date = "0100-01-01"

climo_nyears = 35

lc_kwargs = {}

hist = 0

In [ ]:
cupid_run = 1

if cupid_run == 1:

    from dask.distributed import Client, LocalCluster

    # Spin up cluster (if running in parallel)
    # client = None
    if not serial:
        cluster = LocalCluster(**lc_kwargs)
        client = Client(cluster)

    client

else:

    from dask.distributed import Client
    from dask_jobqueue import PBSCluster

    cluster = PBSCluster(
        cores=16,
        processes=16,
        memory="100GB",
        account="P93300065",
        queue="casper",
        walltime="02:00:00",
    )

    client = Client(cluster)

    cluster.scale(1)

    print(cluster)
    print(client)

In [ ]:
# Read in two cases. The ADF timeseries are needed here.

ds1 = xr.open_mfdataset(
    CESM_output_dir
    + "/"
    + case_name
    + "/ice/proc/tseries/month_1/"
    + case_name
    + ".cice.h."
    + "*.nc",
    data_vars="minimal",
    compat="override",
    coords="minimal",
).sel(time=slice(start_date, end_date))
ds2 = xr.open_mfdataset(
    CESM_output_dir
    + "/"
    + base_case_name
    + "/ice/proc/tseries/month_1/"
    + base_case_name
    + ".cice.h."
    + "*.nc",
    data_vars="minimal",
    compat="override",
    coords="minimal",
).sel(time=slice(base_start_date, base_end_date))

ds_grid = xr.open_dataset("/glade/campaign/cesm/community/omwg/grids/tx2_3v2_grid.nc")
TLAT = ds_grid["TLAT"]
TLON = ds_grid["TLONG"]
tarea = ds_grid["TAREA"] * 1.0e-4
angle = ds_grid["ANGLE"]

ds1_ann = ds1.resample(time="YS").mean(dim="time")
ds2_ann = ds2.resample(time="YS").mean(dim="time")

with open("cice_masks.yml", "r") as file:
    cice_masks = yaml.safe_load(file)

with open("cice_vars.yml", "r") as file:
    cice_vars = yaml.safe_load(file)

In [ ]:
from plot_diff import plot_diff
from vect_diff import vect_diff

In [ ]:
for var in cice_vars:
    vmin = cice_vars[var][0]["levels"][0]
    vmax = cice_vars[var][0]["levels"][-1]
    levels = np.array(cice_vars[var][0]["levels"])
    title = cice_vars[var][1]["title"]
    field1 = ds1_ann[var].isel(time=slice(-climo_nyears, None)).mean("time").squeeze()
    field2 = ds2_ann[var].isel(time=slice(-climo_nyears, None)).mean("time").squeeze()
    plot_diff(field1, field2, levels, case_name, base_case_name, title, "N", TLAT, TLON)

In [ ]:
for var in cice_vars:
    vmin = cice_vars[var][0]["levels"][0]
    vmax = cice_vars[var][0]["levels"][1]
    levels = np.array(cice_vars[var][0]["levels"])
    title = cice_vars[var][1]["title"]
    field1 = ds1_ann[var].isel(time=slice(-climo_nyears, None)).mean("time").squeeze()
    field2 = ds2_ann[var].isel(time=slice(-climo_nyears, None)).mean("time").squeeze()
    plot_diff(field1, field2, levels, case_name, base_case_name, title, "S", TLAT, TLON)

In [ ]:
uvel1 = ds1_ann["uvel"].isel(time=slice(-climo_nyears, None)).mean("time").squeeze()
vvel1 = ds1_ann["vvel"].isel(time=slice(-climo_nyears, None)).mean("time").squeeze()
uvel2 = ds2_ann["uvel"].isel(time=slice(-climo_nyears, None)).mean("time").squeeze()
vvel2 = ds2_ann["vvel"].isel(time=slice(-climo_nyears, None)).mean("time").squeeze()

vect_diff(uvel1, vvel1, uvel2, vvel2, angle, "N", case_name, base_case_name, TLAT, TLON)

In [ ]:
vect_diff(uvel1, vvel1, uvel2, vvel2, angle, "S", case_name, base_case_name, TLAT, TLON)